## Prerequisite Installations

In [ ]:
#!pip install pygame

## Importing Packages

In [1]:
import pygame
import math
import random
from pygame import mixer

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Initializing pygame
pygame.init()

(6, 0)

In [3]:
# Creating a window with height and weight dimensions
screen = pygame.display.set_mode((800, 600))

#Background
background = pygame.image.load('bg.png')

#Backgroun sound
mixer.music.load('background.wav')
mixer.music.play(-1)

#Title and Icon(32x32 bit PNG icon)
pygame.display.set_caption("Chicken Invaders")
icon = pygame.image.load('chick.png')
pygame.display.set_icon(icon)

#Player
playerImg = pygame.image.load('play.png')
playerX = 370
playerY = 480
playerX_change = 0

#Enemy
enemyImg = []
enemyX = []
enemyY = []
enemyX_change = [] 
enemyY_change = []
no_of_enemies = 7

for i in range(no_of_enemies):
    enemyImg.append(pygame.image.load('chicken.png'))
    enemyX.append(random.randint(0,735))
    enemyY.append(random.randint(0,100))
    enemyX_change.append(0.6)
    enemyY_change.append(40)

#Bullet
bulletImg = pygame.image.load('bullet.png')
bulletX = 0
bulletY = 480
bulletX_change = 0
bulletY_change = 5
bullet_state = "ready"  #Ready: You can't see the bullet on screen
#Fire: The bullet is currently moving

#Score
score = 0
font = pygame.font.Font('freesansbold.ttf', 32)

testX = 10
testY = 10

#GameOver Text
over_font = pygame.font.Font('freesansbold.ttf', 64)


def show_score(x,y):
    final_score = font.render("Score:" + str(score), True, (255,255,255))
    screen.blit(final_score, (x,y))

def game_over_text():
    over_text = over_font.render("Game Over!!", True, (255,255,255))
    screen.blit(over_text, (200,250))
    
def player(x,y):
    '''
    blit: draws an image on the screen
    '''
    screen.blit(playerImg, (round(x),round(y)))      
    
def enemy(x,y,i):
    screen.blit(enemyImg[i], (round(x),round(y)))
    
def fire_bullet(x,y):
    global bullet_state
    bullet_state = "fire"
    screen.blit(bulletImg, (x+16 , y+10))    

def isCollision(enemyX, enemyY, bulletX, bulletY):
    distance = math.sqrt((math.pow(enemyX-bulletX,2)) + (math.pow(enemyY-bulletY,2)))
    if distance < 27:
        return True
    else:
        return False
    
#For the window to hold on and quit when close event occurs - Game loop
running = True
while running:
    
    #RGB - Red, Green, Blue  (For reference: https://www.rapidtables.com/convert/color/hex-to-rgb.html)
    screen.fill((0,0,0))
    #Background Image
    screen.blit(background, (0,0))
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
        # pressing keystrokes for the event to occur
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                playerX_change = -2
            if event.key == pygame.K_RIGHT:
                playerX_change = 2
            if event.key == pygame.K_SPACE:
                if bullet_state == "ready":
                    bullet_Sound = mixer.Sound('fire.wav')
                    bullet_Sound.play()
                    bulletX = playerX         #current x cordinate
                    fire_bullet(bulletX, bulletY)
        
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                playerX_change = 0
    
    #Boundaries for player
    playerX += playerX_change
    
    if playerX <=0:
        playerX = 0
    elif playerX >=736:
        playerX = 736
    
    #Boundaries for enemy
    for i in range(no_of_enemies):
        
        #Game Over
        if enemyY[i] > 440:
            for j in range(no_of_enemies):
                enemyY[j] = 2000
            game_over_text()
            break
        
        enemyX[i] += enemyX_change[i]
        if enemyX[i] <=0:
            enemyX_change[i] = 0.6     #hitting on left boudary starts moving towards right
            enemyY[i] += enemyY_change[i]
        elif enemyX[i] >=736:
            enemyX_change[i] = -0.6
            enemyY[i] += enemyY_change[i]
            
        #Collision
        collision = isCollision(enemyX[i], enemyY[i], bulletX, bulletY)
        if collision:
            explosion_Sound = mixer.Sound('kill.wav')
            explosion_Sound.play()
            bulletY = 480
            bullet_state = "ready"
            score += 1
            enemyX[i] = random.randint(0,735)
            enemyY[i] = random.randint(0,100)
            
        enemy(enemyX[i], enemyY[i], i)

    #Bullet Movement
    if bulletY <=0:
        bulletY =480
        bullet_state = "ready"
    
    if bullet_state == "fire":
        fire_bullet(bulletX, bulletY)
        bulletY -= bulletY_change
                
    player(playerX, playerY)
    show_score(testX,testY)
    pygame.display.update()